# <center><font color='blue'>SENTIMENT ANALYSIS: COVID</center></font>

## Table of contents
- [1 - Objetivos](#1)
- [2 - Librerías necesarias](#2)
- [3 - Carga y visualización de datos](#3)
- [4 - Pre-procesamiento de datos](#4)
    - [4.1. - Datos faltantes](#4.1)
    - [4.2. - Data Categóricos](#4.2)
    - [4.3. - Balanceo de clases](#4.3)
    - [4.4. - Pre-Procesamiento especial para NLP](#4.4)
- [5 - Modelos](#5)
    - [5.1. - Modelo 1](#5.1)
    - [5.2. - Modelo 2](#5.2)
    - [5.3. - Modelo 3](#5.3)
    - [5.4. - Modelo 4](#5.4)
    - [5.4. - Modelo 5](#5.5)
    - [5.4. - Modelo 6](#5.6)
    - [5.4. - Modelo 7](#5.7)
- [6 - Comparando los modelos y eligiendo el mejor](#6)
- [7 - Ajuste de hiperparámetros](#7)
- [8 - Predicciones con el modelo final](#8)
- [9 - Guardando el modelo](#9)
- [10 - Conclusiones](#10)

<a name="1"></a>
## <b> <font color='blue'> 1. Objectives </font> </b>

Practice with a natural language processing problem.
<br>
Here, given a set of tweets, analyze whether the sentiment is positive, negative, or neutral.

<a name="2"></a>
## <b> <font color='blue'> 2. Setup </font> </b>

In [1]:
# do not print info and warning messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
# basic imports
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

2024-07-19 21:05:02.858003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 21:05:02.858036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 21:05:02.859219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<a name="3"></a>
## <b> <font color='blue'> 3. Data loading and visualization </font> </b>

In [3]:
!ls data

Corona_NLP_test.csv  Corona_NLP_train.csv


We can see that we have 2 datasets, one for training and one for test.

In [4]:
train_data = pd.read_csv('data/Corona_NLP_train.csv',encoding='latin-1')
test_data = pd.read_csv('data/Corona_NLP_test.csv',encoding='latin-1')

In [5]:
# let's see some of the training data
pd.set_option('display.max_colwidth', None) # que No recorte el texto

train_data.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,"As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, @Tim_Dodson reports https://t.co/cfXch7a2lU",Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,"Cashier at grocery store was sharing his insights on #Covid_19 To prove his credibility he commented ""I'm in Civics class so I know what I'm talking about"". https://t.co/ieFDNeHgDO",Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toilet paper. #Rebel\r\r\n\r\r\n#toiletpapercrisis #covid_19 https://t.co/eVXkQLIdAZ,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,"Due to COVID-19 our retail store and classroom in Atlanta will not be open for walk-in business or classes for the next two weeks, beginning Monday, March 16. We will continue to process online and phone orders as normal! Thank you for your understanding! https://t.co/kw91zJ5O5i",Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy things with the cash and should use online payment methods because corona can spread through the notes. Also we should prefer online shopping from our home. It's time to fight against COVID 19?. #govindia #IndiaFightsCorona",Negative


In [6]:
print(f" Training data: (rows, cols) = {train_data.shape}")
print(f" Training data type:  {type(train_data)}")
print(f" Test data: (rows, cols) = {test_data.shape}")
print(f" Test data type:  {type(test_data)}")

 Training data: (rows, cols) = (41157, 6)
 Training data type:  <class 'pandas.core.frame.DataFrame'>
 Test data: (rows, cols) = (3798, 6)
 Test data type:  <class 'pandas.core.frame.DataFrame'>


<a name="4"></a>
## <b> <font color='blue'> 4. Data Pre-processing </font> </b>


<a name="4.1"></a>
### <b> <font color='#5499C7'> 4.1. Handling missing data </font> </b>

In [7]:
print(f'Missing data train:\n {train_data.isnull().sum()} \n')
print(f'Missing data test:\n {test_data.isnull().sum()}')

Missing data train:
 UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64 

Missing data test:
 UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64


There are no missing data in the columns we care about (Original Tweet and Sentiment).

<a name="4.2"></a>
### <b> <font color='#5499C7'> 4.2. Separate features from target </font> </b>

In [8]:
X_train = train_data['OriginalTweet']
y_train = train_data['Sentiment']
X_test = test_data['OriginalTweet']
y_test = test_data['Sentiment']

In [9]:
type(X_train), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

<a name="4.3"></a>
### <b> <font color='#5499C7'> 4.3. Categorical data </font> </b>

Our target (Sentiment) is categorical, lets see the different options:

In [10]:
y_train.unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

Lets see if there are other options in the test set:

In [11]:
y_test.unique()

array(['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative',
       'Neutral'], dtype=object)

In [12]:
are_same = np.array_equal(np.sort(y_train.unique()), np.sort(y_test.unique()))
print(f"Are unique values the same? {are_same}")

Are unique values the same? True


We are not so interested in whether a sentiment is positive or extremely positive, therefore we will rename the label extremely positive to positive.

In [13]:
y_train = y_train.replace('Extremely Positive', 'Positive').replace('Extremely Negative', 'Negative')
y_test = y_test.replace('Extremely Positive', 'Positive').replace('Extremely Negative', 'Negative')

Check

In [14]:
print(np.sort(y_train.unique()), np.sort(y_test.unique()))

['Negative' 'Neutral' 'Positive'] ['Negative' 'Neutral' 'Positive']


We will not consider that there is an ordinal relationship, therefore we will encode the labels using one-hot encoding.

Remember that we need to apply the same encoding to train and test.

In [15]:
from sklearn.preprocessing import OneHotEncoder


# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit the encoder on the train set
encoder.fit(y_train.values.reshape(-1, 1))

# Transform both train and test sets
one_hot_train = encoder.transform(y_train.values.reshape(-1, 1))
one_hot_test = encoder.transform(y_test.values.reshape(-1, 1))

# Convert to DataFrame for easier inspection
y_train = pd.DataFrame(one_hot_train, columns = encoder.get_feature_names_out(['sentiment']))
y_test = pd.DataFrame(one_hot_test, columns = encoder.get_feature_names_out(['sentiment']))

print("One-hot encoded train set:")
print(y_train)
print("\nOne-hot encoded test set:")
print(y_test)

One-hot encoded train set:
       sentiment_Negative  sentiment_Neutral  sentiment_Positive
0                     0.0                1.0                 0.0
1                     0.0                0.0                 1.0
2                     0.0                0.0                 1.0
3                     0.0                0.0                 1.0
4                     1.0                0.0                 0.0
...                   ...                ...                 ...
41152                 0.0                1.0                 0.0
41153                 1.0                0.0                 0.0
41154                 0.0                0.0                 1.0
41155                 0.0                1.0                 0.0
41156                 1.0                0.0                 0.0

[41157 rows x 3 columns]

One-hot encoded test set:
      sentiment_Negative  sentiment_Neutral  sentiment_Positive
0                    1.0                0.0                 0.0
1           

In this example, handle_unknown='ignore' is used to ensure that any categories present in the test set but not in the train set are handled gracefully. This is useful when you have categories in the test set that were not seen during training.

<a name="4.4"></a>
### <b> <font color='#5499C7'> 4.4. Train/val split </font> </b>

We are going to create a validation dataset.

In [16]:
from sklearn.model_selection import train_test_split 

# Perform the split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


<a name="4.5"></a>
### <b> <font color='#5499C7'> 4.5. Class balance </font> </b>

In [17]:
y_train.value_counts(normalize=True)

sentiment_Negative  sentiment_Neutral  sentiment_Positive
0.0                 0.0                1.0                   0.438238
1.0                 0.0                0.0                   0.374670
0.0                 1.0                0.0                   0.187092
dtype: float64

We have some imbalance with respect to the neutral class. We are going to apply SMOTE to take care of this.

In [18]:
#!pip install imbalanced-learn

In [19]:
"""
from imblearn.over_sampling import SMOTE

# Applyt SMOTE only to the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Show the new distribution
print(y_train_resampled.value_counts())

para smote debo tener las etiquetas en formato categórico

"""

'\nfrom imblearn.over_sampling import SMOTE\n\n# Applyt SMOTE only to the training set\nsmote = SMOTE(random_state=42)\nX_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)\n\n# Show the new distribution\nprint(y_train_resampled.value_counts())\n\npara smote debo tener las etiquetas en formato categórico\n\n'

<a name="4.6"></a>
### <b> <font color='#5499C7'> 4.6. Pre-process for NLP </font> </b>

Let's preprocess the text from OriginalTweet, for which:

- We will remove stop words.
- We will remove certain special characters, like "@".
- We will apply lemmatization.

<b>Note:</b> We will also remove punctuation, convert everything to lowercase, and tokenize later using TextVectorization.


In [20]:
from utils import preprocess_text

[nltk_data] Downloading package stopwords to /home/marcos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marcos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marcos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Let's try our function:

In [21]:
text = "Hello @world! This is# a sample text."
processed_text = preprocess_text(text)
print(processed_text)

hello world ! sample text .


Let's apply ou pre-process function:

In [22]:
X_train_prep = X_train.apply(preprocess_text)
X_val_prep = X_val.apply(preprocess_text)
X_test_prep = X_test.apply(preprocess_text)

Let' see a few examples:

In [23]:
X_train[0]

'@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8'

In [24]:
X_train_prep[0]

'menyrbie phil_gahan chrisitv'

In [25]:
X_train.index

Int64Index([ 8191,  3725, 22759, 15010, 39142, 30040, 30389, 18537,   919,
            33677,
            ...
            41090, 16023, 21962, 37194, 16850,  6265, 11284, 38158,   860,
            15795],
           dtype='int64', length=32925)

In [26]:
X_train[8191]

'Unemployment claims made online in Virginia this week:\r\r\n\r\r\nMonday: 426\r\r\nTuesday: 2,150\r\r\n\r\r\nAnd the numbers are going to get bigger. https://t.co/fUeg2RL2dl'

In [27]:
X_train_prep[8191]

'unemployment claim made online virginia week : monday : 426 tuesday : 2,150 number going get bigger .'

### Text Vectorization layer

Now we will create and apply a layer called <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization" target='_blank'>TextVectorization</a>, which will remove punctuation, convert everything to lowercase, and tokenize.

In [28]:
VOCAB_SIZE = 10000
sequence_length = 50 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)


vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE,
                                    output_mode="int",
                                    output_sequence_length = sequence_length)

# Fit the text vectorizer instance to the training data using the adapt() method
vectorizer.adapt(X_train_prep)

Let' see the 20 firsts tokens:

In [29]:
vocab = np.array(vectorizer.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'coronavirus', 'covid19', 'price', 'store',
       'supermarket', 'food', 'grocery', 'people', 'amp', 'consumer',
       '19', 'covid', 'shopping', 's', 'online', 'need', 'time',
       'pandemic'], dtype='<U27')

We are going to apply this layer to our datasets.

In [30]:
# apply to the datasets
X_train_vectors = vectorizer(X_train_prep)
X_val_vectors = vectorizer(X_val_prep)
X_test_vectors = vectorizer(X_test_prep)

In [31]:
X_train_vectors

<tf.Tensor: shape=(32925, 50), dtype=int64, numpy=
array([[ 821,  743,  232, ...,    0,    0,    0],
       [ 269,  189,    3, ...,    0,    0,    0],
       [   1, 2136, 5977, ...,    0,    0,    0],
       ...,
       [   1,    1, 2221, ...,    0,    0,    0],
       [1033,  127, 1425, ...,    0,    0,    0],
       [1361, 8579,    2, ...,    0,    0,    0]])>

<a name="4.7"></a>
### <b> <font color='#5499C7'> 4.7. Prepare datasets for TensorFlow </font> </b>

In [32]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_vectors, 
                                              y_train.values))


validation_dataset = tf.data.Dataset.from_tensor_slices((X_val_vectors, 
                                              y_val.values))


test_dataset = tf.data.Dataset.from_tensor_slices((X_test_vectors, 
                                              y_test.values))


In [33]:
# let's see an example for train
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  [ 821  743  232   16 3366   47  806    1 1277    1  297   35   20 2672
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
label:  [1. 0. 0.]


We define the size of the buffer and the size of the batch:

In [34]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32

In [35]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Let's see some examples and their labels

In [36]:
for example, label in train_dataset.take(1):
  print(f'texts:  {example.numpy()[:3]}\n')
  print(f'labels: , {label.numpy()[:3]}')

texts:  [[   1 1216  663  478  801  663  478  285  209  663  478 2913 4714    4
   663  478  195   22  225   46    3    2    1    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [  19  766  143  364  868  937  161    3   42  766  143  868  364  169
   294   11 1074  131   14   58   78 1902 9468    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [  46  879    1    1    7  117  560  329  230   30  385  687   34   46
    32  251  882  283 1318 1159   86  151  256    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]

labels: , [[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


<a name="5"></a>
## <b> <font color='blue'> 5. Models </font> </b>

Let's try different models; all of them are going to have an embedding layer, that's why we are going to start by creating one of them.

Also, we're gonna set the random seed, create a dict to store the results, define a constant with the size of the input and a constant with the number of classes.

In [37]:
tf.random.set_seed(42)

results = {}

INPUT_SHAPE=(sequence_length,)

NUM_CLASSES = 3

In [38]:
# imports
from tensorflow.keras import layers,callbacks,models,Sequential,losses

# function to store the results
from utils import store_results

<a name="5.1"></a>
### <b> <font color='#5499C7'> 5.1. Embedding layer </font> </b>

In [39]:
embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, # set the input shape; size of our vocabulary
                                 output_dim=128, # set the size of the embedding vector
                                 embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                                 input_length=sequence_length # how long is each input
                             )

embedding

<a name="5.2"></a>
### <b> <font color='#5499C7'> 5.2. Compile and fit the models </font> </b>

"Since it will be the same for everyone, we define a function to compile and train the models."


In [40]:
def compile_fit(model, train_dataset, validation_dataset, 
                    epochs=5, loss='categorical_crossentropy', optimizer='adam',
                    metrics=['accuracy', 'Precision','Recall'], verbose = 1):
    # Compile model
    model.compile(loss=loss,
                    optimizer = optimizer,
                    metrics = metrics)

    # Fit the model
    history = model.fit(train_dataset,
                            epochs = epochs,
                            verbose = verbose,
                            validation_data = validation_dataset)
    
    # return
    return history

<a name="5.2"></a>
### <b> <font color='#5499C7'> 5.2. Model 1 </font> </b>

In [41]:
def build_model_1(input_shape, num_classes, name):
    inputs = layers.Input(shape=input_shape) # inputs are 1-dimensional integers
    x = embedding(inputs)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs, name=name) # construct the model
    return model


model_1 = build_model_1(INPUT_SHAPE, NUM_CLASSES, 'model_1')


In [42]:
model_1.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 1280387 (4.88 MB)
Trainable params: 1280387 (4.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
history_1 = compile_fit(model_1, train_dataset, validation_dataset)

Epoch 1/5
1029/1029 [==============================] - 21s 20ms/step - loss: 0.8785 - accuracy: 0.6111 - precision: 0.8124 - recall: 0.2937 - val_loss: 0.7150 - val_accuracy: 0.7287 - val_precision: 0.8216 - val_recall: 0.5573
Epoch 2/5
1029/1029 [==============================] - 20s 20ms/step - loss: 0.5877 - accuracy: 0.7890 - precision: 0.8550 - recall: 0.6794 - val_loss: 0.5993 - val_accuracy: 0.7824 - val_precision: 0.8375 - val_recall: 0.7019
Epoch 3/5
1029/1029 [==============================] - 21s 20ms/step - loss: 0.4604 - accuracy: 0.8492 - precision: 0.8868 - recall: 0.7936 - val_loss: 0.5766 - val_accuracy: 0.7931 - val_precision: 0.8289 - val_recall: 0.7340
Epoch 4/5
1029/1029 [==============================] - 24s 23ms/step - loss: 0.3896 - accuracy: 0.8762 - precision: 0.9013 - recall: 0.8429 - val_loss: 0.5639 - val_accuracy: 0.8020 - val_precision: 0.8299 - val_recall: 0.7688
Epoch 5/5
1029/1029 [==============================] - 24s 23ms/step - loss: 0.3430 - accura

In [44]:
# evaluate
score1 = model_1.evaluate(test_dataset)

119/119 [==============================] - 0s 1ms/step - loss: 0.6032 - accuracy: 0.7894 - precision: 0.8075 - recall: 0.7575


In [51]:
# save the results
store_results(results, 'Model 1', score1); # ";" because i do not want to print the output

<a name="5.3"></a>
### <b> <font color='#5499C7'> 5.3. Model 2 - LSTM </font> </b>

In [ ]:
def build_model_2(input_shape, num_classes, name):
    inputs = layers.Input(shape=input_shape, dtype="float32")
    x = embedding(inputs)
    x = layers.LSTM(64, activation="tanh")(x)
    x = layers.Dropout(0.5)(x) 
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs, name=name)
    return model
    
model_2 = build_model_2(INPUT_SHAPE, NUM_CLASSES, 'model_2')


In [ ]:
model_2.summary()

In [ ]:
history_2 = compile_fit(model_2, train_dataset, validation_dataset, verbose = 0)

In [ ]:
# evaluate
score2 = model_2.evaluate(test_dataset)

In [ ]:
# save the results
store_results(results, 'Model 2', score2)

<a name="5.4"></a>
### <b> <font color='#5499C7'> 5.4. Model 3 - GRU </font> </b>

In [ ]:
def build_model_3(input_shape, num_classes, name):
    inputs = layers.Input(shape=(1,), dtype="float32")
    x = embedding(inputs)
    x = layers.GRU(64, activation="tanh")(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model= tf.keras.Model(inputs, outputs, name=name)
    return model


model_3 = build_model_3(INPUT_SHAPE, NUM_CLASSES, 'model_3')

In [ ]:
model_3.summary()

In [ ]:
history_3 = compile_fit(model_3, train_dataset, validation_dataset, verbose = 0)

In [ ]:
# evaluate
score3 = model_3.evaluate(test_dataset)

In [ ]:
# save the results
store_results(results, 'Model 3', score3)

<a name="5.5"></a>
### <b> <font color='#5499C7'> 5.5. Model 4 - Bidirectional RNN </font> </b>

In [ ]:
def build_model_4(input_shape, num_classes, name):
    inputs = layers.Input(shape=input_shape, dtype="float32")
    x = embedding(inputs)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs, name=name)
    return model

model_4 = build_model_4(INPUT_SHAPE, NUM_CLASSES, 'model_4')

In [ ]:
model_4.summary()

In [ ]:
history_4 = compile_fit(model_4, train_dataset, validation_dataset, verbose = 0)

In [ ]:
# evaluate
score4 = model_4.evaluate(test_dataset)

In [ ]:
# save the results
store_results(results, 'Model 4', score4)

<a name="5.6"></a>
### <b> <font color='#5499C7'> 5.6. Model 5 - Stacking layers </font> </b>

In [ ]:
def build_model_5(input_shape, num_classes, name):
    inputs = layers.Input(shape=input_shape, dtype='float32')
    x = embedding(inputs)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # return_sequences=True to stack layers
    x = layers.Bidirectional(layers.LSTM(32))(x)
    x = layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs, name=name)
    return model
    

model_5 = build_model_5(INPUT_SHAPE, NUM_CLASSES, 'model_5')

model_5.summary()


In [ ]:
history_5 = compile_fit(model_5, train_dataset, validation_dataset, verbose = 0)

In [ ]:
# evaluate
score5 = model_5.evaluate(test_dataset)

In [ ]:
# save the results
store_results(results, 'Model 5', score5)

<a name="5.7"></a>
### <b> <font color='#5499C7'> 5.7. Model 6 - Conv 1D </font> </b>

We've seen before how convolutional neural networks can be used for images but they can also be used for text.

Previously we've used the layer Conv2D (which is great for images with (height, width)).

But if we want to use convolutional layers for sequences (e.g. text) we need to use Conv1D: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D

For more of a deep dive into what goes on behind the scenes in a CNN for text (or sequences) see the paper: https://arxiv.org/abs/1809.08037


In [ ]:
def build_model_6(input_shape, num_classes, name):
    inputs = layers.Input(shape = input_shape)    
    x = embedding(inputs)
    x = layers.Conv1D(filters = 32, kernel_size= 5, activation="relu")(x)
    x = layers.GlobalMaxPool1D()(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)
    model = tf.keras.Model(inputs, outputs, name=name)
    return model
    
    
model_6 = build_model_6(INPUT_SHAPE, NUM_CLASSES, 'model_6')

model_6.summary()

In [ ]:
history_6 = compile_fit(model_6, train_dataset, validation_dataset, verbose = 0)

In [ ]:
# evaluate
score6 = model_6.evaluate(test_dataset)

In [ ]:
# save the results
store_results(results, 'Model 6', score6)

<a name="5.7"></a>
### <b> <font color='#5499C7'> 5.7. Model 7 - TensorFlow Hub Pretrained Sentence Encoder </font> </b>

Now we've built a few of our own models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder: https://tfhub.dev/google/universal-sentence-encoder/4

See how the USE was created here: https://arxiv.org/abs/1803.11175

📖 **Resource:** TensorFlow Hub is a great resource for many pretrained models but HuggingFace is also another incredible resource for many pretrained NLP models (using HuggingFace model is beyond the scope of this course but it is definitely something you should be familiar with in the NLP space): https://huggingface.co/models

In [ ]:
import tensorflow_hub as hub

In [ ]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
# it takes care of text vectorization and embedding

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [ ]:
#https://stackoverflow.com/questions/47268608/confusion-about-keras-rnn-input-shape-requirement
#INPUT_SHAPE = ()  # Scalar for raw text input


def build_model_7(input_shape, num_classes, name):
    inputs = layers.Input(shape=(), dtype=tf.string)  # Expect raw text input
    x = sentence_encoder_layer(inputs)  # Encode text to embeddings
    x = layers.Dense(128, activation='relu')(x)  # Fully connected layer
    x = layers.Dropout(0.5)(x)  # Optional dropout layer
    outputs = layers.Dense(num_classes, activation='softmax', name='output_layer')(x)
    model = models.Model(inputs, outputs, name=name)
    return model

model_7 = build_model_7(INPUT_SHAPE, NUM_CLASSES, 'model_7')

model_7.summary()

In [ ]:
# data for this model (remember this model does text vectorization)
train_dataset2 = tf.data.Dataset.from_tensor_slices((X_train_prep, 
                                              y_train.values))


validation_dataset2 = tf.data.Dataset.from_tensor_slices((X_val_prep, 
                                              y_val.values))


test_dataset2 = tf.data.Dataset.from_tensor_slices((X_test_prep, 
                                              y_test.values))


In [ ]:
train_dataset2 = train_dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset2 = validation_dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset2 = test_dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
history_7 = compile_fit(model_7, train_dataset2, validation_dataset2, verbose = 1)

In [ ]:
# evaluate
score7 = model_7.evaluate(test_dataset2)

In [ ]:
# save the results
store_results(results, 'Model 7', score7)

<a name="5.7"></a>
### <b> <font color='#5499C7'> 5.7. Choosing the best model </font> </b>

In [ ]:
results

In [ ]:
# Convert dictionary to DataFrame
df_metrics = pd.DataFrame(results).T  # .T transposes the DataFrame for easier viewing
df_metrics.reset_index(inplace=True)
df_metrics.rename(columns={'index': 'Model'}, inplace=True)

# Display the DataFrame
df_metrics

Let's plot the results:

In [ ]:
# Extract data
models = list(results.keys())
metrics = list(results[models[0]].keys())
values = {metric: [] for metric in metrics}

for model in models:
    for metric in metrics:
        values[metric].append(results[model][metric])

x = np.arange(len(models))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(8, 6))

# Plot each metric
for i, metric in enumerate(metrics):
    ax.bar(x + i * width, values[metric], width, label=metric)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Models', fontweight='bold')
ax.set_ylabel('Value', fontweight='bold')
ax.set_title('Comparison of Models by Metrics', fontweight='bold', color='#12222C')
ax.set_xticks(x + width * (len(metrics) / 2 - 0.5))
ax.set_xticklabels(models)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Now, let's view each metric separately:

In [ ]:
fig, axs = plt.subplots(len(metrics), 1, figsize=(8, 6), sharex=True)

for i, metric in enumerate(metrics):
    values = [results[model][metric] for model in models]
    axs[i].bar(models, values, color=['b', 'g', 'r'])
    axs[i].set_title(f'{metric.capitalize()} Comparison', fontweight='bold', color='#1B4F72')
    axs[i].set_ylabel('Value', fontweight='bold')

axs[-1].set_xlabel('Models', fontweight='bold')

plt.tight_layout()
plt.show()


To choose the best model, we will use the F1-score as our metric.

It should be noted that this is a simplification, as we are only training for 5 epochs and not considering other factors, such as which models are more prone to overfitting, which ones take longer to train and infer, etc."

In [ ]:
# Find the index of the row with the highest F1 score
highest_f1_index = df_metrics['f1_score'].idxmax()

# Get the model with the highest F1 score
best_model = df_metrics.loc[highest_f1_index]

print(f"Model with the highest F1 score:\n{best_model}")

<a name="6"></a>
## <b> <font color='blue'> 6. Hyperparamter Tunning </font> </b>